![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg)

<center>
<h1><font size="+3">GSFC Python Bootcamp</font></h1>
</center>

---

<CENTER>
<H1 style="color:red">
An Introduction to netCDF4 Python
</H1>
</CENTER>

In [ ]:
from __future__ import print_function

# <font color='red'> Useful References </font>

* <A HREF="http://pyhogs.github.io/intro_netcdf4.html">Create and read netCDF files</A>
* <A HREF="https://unidata.github.io/netcdf4-python/netCDF4/index.html">netCDF4 module</A>

## <font color="red"> What we will cover </font>
* Opening a file
* Dimension
* Variables
* Attributes
* Writing data
* Creating groups
* Reading data

## <font color='red'> What is netCDF?</font>

**Overview**

* The Network Common Data Form, or netCDF, is an interface to a library of data access functions for storing and retrieving data in the form of arrays.
* NetCDF is an abstraction that supports a view of data as a collection of self-describing, portable objects that can be accessed through a simple interface.
* All operations to access and manipulate data in a netCDF dataset must use only the set of functions provided by the interface.
* Array values may be accessed directly, without knowing details of how the data are stored.
* NetCDF supports efficient access to small subsets of large datasets.

**Portability**

* The netCDF library is supported for various Linux/UNIX operating systems as well as MS Windows.
* APIs written for Fortran 77/90, C, C++, Java

**Conventions**

* The mere use of netCDF is not sufficient to make data "self-describing" and meaningful to both humans and machines. 
* By using a set of conventions, a data producer is more likely to produce files that can be easily shared within the research community, and that contain enough details to be useful as a long-term archive. 
* The names of variables and dimensions should be meaningful and conform to any relevant conventions.
* It is important to use all the relevant standard attributes using the relevant conventions.



## <font color='red'> What is netCDF4 Python?</font>

* Python interface to the netCDF version 4 library.
* **Can read and write files in both the new netCDF 4 and the netCDF 3 formats**.
* Can create files that are readable by HDF5 utilities.
* Relies on NumPy arrays.

**<font color="blue">Run the cell below only if you are in Google Colab</font>**

In [ ]:
!pip install netCDF4

-----

In [ ]:
import numpy as np
from netCDF4 import Dataset

#### <font color='red'> Opening a netCDF File</font>

In [ ]:
ncFileName = 'sample_netcdf.nc4'
modeType   = 'w'
fileFormat = 'NETCDF4'
ncfid = Dataset(ncFileName, mode=modeType, format=fileFormat)

`modeType` has the options:
* 'w': to create a new file
* 'r+': to read and write with an existing file
* 'r': to read (only) an existing file
* 'a': to append to existing file

`fileFormat` has the options: 
* 'NETCDF3_CLASSIC': Original netCDF format     
* 'NETCDF3_64BIT_OFFSET': Used to ease the size restrictions of netCDF classic files
* 'NETCDF4_CLASSIC'
* 'NETCDF4': Offer new features such as groups, compound types, variable length arrays, new unsigned integer types, parallel I/O access, etc.
* 'NETCDF3_64BIT_DATA'

#### <font color='red'> Creating Dimensions in a netCDF File</font>
* Use the method `createDimension`
* It typically takes as arguments a string (name of the dimension) and an integer (dimension size)
* For unlimited dimensions, use `None` as size.
* Unlimited size dimensions must be declared before (“to the left of”) other dimensions.

In [ ]:
time = ncfid.createDimension('time', None)
lev  = ncfid.createDimension('lev', 72)
lat  = ncfid.createDimension('lat', 91)
lon  = ncfid.createDimension('lon', 144)

#### <font color='red'>Create Variables</font>
* Use the method ``createVariable``
* The first three arguments are the variable name, the variable type, a tuple of previously defined dimension names.
* If you need data compression, add the `zlib=True` argument.

**Dimension Variables**

In [ ]:
times      = ncfid.createVariable('time','f8',('time',))
levels     = ncfid.createVariable('lev','i4',('lev',))
latitudes  = ncfid.createVariable('lat','f4',('lat',))
longitudes = ncfid.createVariable('lon','f4',('lon',))

**Regular Variables**

In [ ]:
temp = ncfid.createVariable('temp','f4', \
                            ('time','lev','lat','lon',))

#### <font color='red'>Adding Variable Attributes</font>
* Attributes allow us to capture metadata that would otherwise be separated from the data.
* Variable attributes are added on individual variables in the dataset.
* All variables should have values for the following attributes unless there is a very good reason not to:

        o units
        o long_name

You may also consider attributes such as:
* valid_min: Smallest valid value of a variable.
* valid_max: Largest valid value of a variable.
* _ FillValue: The value that a variable gets filled with before any data is loaded into it.
* standard_name: A name used to identify the physical quantity. A standard name contains no whitespace and is case sensitive. 

In [ ]:
latitudes.long_name  = 'latitude'
latitudes.units      = 'degrees north'

longitudes.long_name = 'longitude'
longitudes.units     = 'degrees east'

levels.long_name     = 'vertical levels'
levels.units         = 'hPa'
levels.positive      = 'down'

times.long_name      = 'time'
times.units          = 'hours since 2019-12-19 11:59:43.0'
times.calendar       = 'gregorian'

temp.long_name       = 'temperature'
temp.units           = 'K'
temp._FillValue      = 1.0e15
temp.valid_min       = 200.
temp.valid_max       = 350.
temp.standard_name   = 'atmospheric_temperature'

#### <font color='red'>Adding Global Attributes</font>
* Global attributes are on the dataset.
* We use directly the file identifier to add them.

In [ ]:
import time
ncfid.description = 'Sample netCDF file'
ncfid.institution = 'NASA GSFC'
ncfid.history     = 'File created on' + time.ctime(time.time())
ncfid.source      = 'netCDF4 python tutorial'

#### <font color='red'>Writing Data in the File</font>

In [ ]:
latitudes[:]  =  np.arange(-90,91,2.0)
longitudes[:] =  np.arange(-180,180,2.5)
levels[:]     =  np.arange(0,72,1)

out_frequency = 3   # ouput frequency in hours
num_records   = 5
for i in range(num_records):
    times[i]      = i*out_frequency
    temp[i,:,:,:] = np.random.uniform(size=(levels.size,
                                            latitudes.size,
                                            longitudes.size))

#### <font color='red'>Printing Dimension Information</font>

In [ ]:
# List all the dimension information
for dim in ncfid.dimensions.values():
     print(dim, dim.isunlimited())

In [ ]:
# Get the list of dimension name and retrieve info for each dimension
for name in ncfid.dimensions.keys():
    dim = ncfid.variables[name]
    print(name, dim.dtype, dim.size)

#### <font color='red'>Printing File Attributes</font>

In [ ]:
# Get the global file attributes
for att in ncfid.ncattrs():
    print(att+':', getattr(ncfid,att))

In [ ]:
# Global attributes as a dictionary
print(ncfid.__dict__)

#### <font color='red'>Printing Variable Information</font>

In [ ]:
# List variable information but exclude dimensions
for name in ncfid.variables.keys():
    if (name not in ncfid.dimensions.keys()):
       data = ncfid.variables[name]
       print(name, data.units, data.shape, data.dtype, data.dimensions)

In [ ]:
def print_ncattr(ncfid, key):
    """
        Prints the NetCDF file attributes for a given key

        Parameters: 
            * ncfid:  netCDF file identifier
            * key:    unicode (a valid netCDF4.Dataset.variables key)
    """
    try:
        print(key, '-->')
        print("\t\ttype:", repr(ncfid.variables[key].dtype))
        for ncattr in ncfid.variables[key].ncattrs():
            print('\t\t%s:' % ncattr,\
                    repr(ncfid.variables[key].getncattr(ncattr)))
    except KeyError:
        print("\t\tWARNING: %s does not contain variable attributes" % key)

In [ ]:
print(print_ncattr.__doc__)

In [ ]:
for name in ncfid.variables.keys():
    print_ncattr(ncfid, name)

#### <font color='red'>Create Groups</font>
* We can organize data in hierarchical groups, which are analogous to directories in a filesystem. 
* Groups serve as containers for variables, dimensions and attributes, as well as other groups.
* Use the method `createGroup` to create groups.

In [ ]:
fcstgrp  = ncfid.createGroup('forecasts')
fcstgrpm = ncfid.createGroup('forecasts/model')

In [ ]:
# List the groups
print(ncfid.groups)

In [ ]:
def walk_group_tree(top):
    """
       Python generator that is used to walk the directory tree.
    """
    values = top.groups.values()
    yield values
    for value in top.groups.values():
        for children in walk_group_tree(value):
            yield children

In [ ]:
# List of the created groups in the dataset
for children in walk_group_tree(ncfid):
    for child in children:
        print(child)

##### Add variable to a group

In [ ]:
tempm = ncfid.createVariable('forecasts/model/temp','f4', \
                                  ('time','lev','lat','lon',))

tempm.long_name       = 'temperature (model)'
tempm.units           = 'K'
tempm._FilledValue    = 1.0e15
tempm.valid_min       = 200.
tempm.valid_max       = 350.
tempm.standard_name   = 'atmospheric_temperature'

tempm[0:num_records,:,:,:] = np.random.uniform(size=(num_records,
                                                     levels.size,
                                                     latitudes.size,
                                                     longitudes.size))

In [ ]:
print(ncfid["forecasts/model"])

In [ ]:
print(ncfid["forecasts/model/temp"])

#### <font color='red'>Close the file</font>

In [ ]:
ncfid.close()

### <font color='red'>Reading a netCDF File</font>

In [ ]:
with Dataset(ncFileName, mode='r') as ncfid:
     time   = ncfid.variables['time'][:]
     lev    = ncfid.variables['lev'][:]
     lat    = ncfid.variables['lat'][:]
     lon    = ncfid.variables['lon'][:]
     temp   = ncfid.variables['temp'][:]
     grpid1 = ncfid.groups['forecasts']
     grpid2 = grpid1.groups['model']
     tempm  = grpid2.variables['temp'][:] 
    
     # Print variable information
     print_ncattr(ncfid,  'time')
     print_ncattr(ncfid,  'temp')
     print_ncattr(grpid2, 'temp')

In [ ]:
print("Time: ", time)

In [ ]:
print("Longitude values: ", lon)

In [ ]:
print("Latitude values: ", lat)

In [ ]:
print("Temperature info: ")
print(temp.shape)
print(np.min(temp), np.max(temp))

In [ ]:
print("Temperature (model) info: ")
print(tempm.shape)
print(np.min(tempm), np.max(tempm))

### <font color='red'>Updating a Variable in an Existing netCDF File</font>

In [ ]:
with Dataset(ncFileName, mode='a') as ncfid:
     temp = ncfid.variables['temp'][:]
     data = temp[:]
     data = 1.1*data + 100.0
     temp[:] = data

In [ ]:
print(temp.shape)
print(np.min(temp), np.max(temp))

## <font color="red">Example</font>

We want to use the netCDF file:

https://www.unidata.ucar.edu/software/netcdf/examples/sresa1b_ncar_ccsm3-example.nc

to plot the surface air temperature (variable `tas`) and the zonal mean height of the wind (variable `ua`).


The metadata of the file is located at:

https://www.unidata.ucar.edu/software/netcdf/examples/sresa1b_ncar_ccsm3-example.cdl

In [ ]:
# Get the remote  file
nc_file = "sresa1b_ncar_ccsm3-example.nc"
url = "https://www.unidata.ucar.edu/software/netcdf/examples/"
import urllib.request
urllib.request.urlretrieve(url+nc_file, nc_file)

In [ ]:
# Open the netCDF file and read surface air temperature
with Dataset(nc_file,'r') as ncid:
     lons      = ncid.variables['lon'][:] # longitude grid points
     lats      = ncid.variables['lat'][:] # latitude grid points
     levs      = ncid.variables['plev'][:] # pressure leves
     surf_temp = ncid.variables['tas'][:]
     uwind     = ncid.variables['ua'][:]

print("Shape of lons:      ", np.shape(lons), lons[0], lons[-1])
print("Shape of lats:      ", np.shape(lats), lats[0], lats[-1])
print("Shape of levs:      ", np.shape(levs), levs[0], levs[-1])
print("Shape of surf_temp: ", np.shape(surf_temp))
print("Shape of uwind:     ", np.shape(uwind))

**Load Plotting Modules**

**<font color='blue'> Only run the cell below if you are on Google Colab</font>**

In [ ]:
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cython
!pip install cartopy

---

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

**Plot of Surface Temperature**

In [ ]:
plt.figure(figsize=(9, 5))
map_projection = ccrs.PlateCarree()
ax = plt.axes(projection=map_projection)
im = ax.contourf(lons, lats, surf_temp[0,:,:], transform=map_projection)
ax.coastlines()

ax.set_xticks(np.linspace(-180, 180, 5), crs=map_projection)
ax.set_yticks(np.linspace(-90, 90, 5), crs=map_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)

plt.colorbar(im, orientation='vertical')

ax.set_global()
plt.show()

**Plot Zonal Mean Height of Wind**

In [ ]:
# Compute the zonal mean height
zonal_mean_height = np.mean(uwind[0,:,:,:], axis=2)

# Select contour levels
ncountours = 10
fac = 0.005
min_val = (1.0-fac)*np.min(zonal_mean_height)
max_val = (1.0+fac)*np.max(zonal_mean_height)
clevs = np.linspace(min_val, max_val, ncountours)

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1)

# map contour values to colors
norm=matplotlib.colors.BoundaryNorm(clevs, ncolors=256, clip=False)
    
# draw the contours with contour labels
CS = ax.contour(lats, levs, zonal_mean_height, levels=clevs)
ax.clabel(CS,inline=1, fontsize=10, colors='black')

# draw the (filled) contours
contour = ax.contourf(lats, levs, zonal_mean_height, levels=clevs, norm=norm)

# Draw colorbar
fmt = matplotlib.ticker.FormatStrFormatter("%3.2g")
cbar = fig.colorbar(contour, ax=ax, orientation='horizontal', shrink=0.8,
                    ticks=clevs, format=fmt)
cbar.set_label('m s-1')

ax.set_yscale('log')
ax.set_xlabel("Latitude (degrees)")
ax.set_ylabel("Pressure (Pa)")
plt.show()